In [ ]:
pip install pmdarima
pip install geopandas

In [ ]:
import numpy as np
import pandas as pd
import os
import json
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import folium 
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from IPython.display import display, Markdown
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20, 16
import warnings
import itertools
warnings.filterwarnings("ignore")
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller


In [ ]:
from google.colab import files
uploaded = files.upload()

데이터 불러오기


In [ ]:
stations = pd.read_csv('Measurement_station_info.csv')
measurements = pd.read_csv('Measurement_info.csv')
items = pd.read_csv('Measurement_item_info.csv')
df = pd.read_csv('Measurement_summary.csv')

데이터 형식 보기

In [ ]:
print('Shape:', items.shape)
items

In [ ]:
df['Station code'].unique()
df

csv의 날짜를 날짜 데이터 형식으로 저장

In [ ]:
df['Measurement date'] = pd.to_datetime(df['Measurement date'])

측정소 코드 109 = 동대문구 천호대로를 분석하기 위해 데이터 추출
101로 다시 수정

In [ ]:
df_109 = pd.DataFrame(df.loc[(df['Station code']==109)])
df_109.head()
df_109.drop("Station code", axis=1, inplace=True)

음수값은 처리

In [ ]:
drop_all = df.loc[(df_109['SO2']<0) | (df['NO2']<0) | (df['CO']<0) | (df['O3']<0)]
drop_PM = df.loc[(df_109['PM2.5']<0) | (df['PM10']<0) | (df['PM2.5']==0) | (df['PM10']==0)]

drop_index = drop_all.index.append(drop_PM.index)
df_new = df.drop(drop_index, axis=0)

날짜 형식 지정 및 결측치 처리 

In [ ]:
df_new['Measurement date'] = pd.datetime(df_new['Measurement date'],format='%Y-%m-%d')
df_new.set_index('Measurement date', drop=True, inplace=True)
df_new.dropna(inplace = True)

In [ ]:
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller

그림 그려줄 함수 정의

In [ ]:
def Plot(ts):
    rol_mean = ts.rolling(window = 12, center = False).mean()
    rol_std = ts.rolling(window = 12, center = False).std()
    
    plt.plot(ts, color = 'blue',label = 'Original Data')
    plt.plot(rol_mean, color = 'red', label = 'Rolling Mean')
    plt.plot(rol_std, color ='black', label = 'Rolling Std')
    plt.xticks(fontsize = 25)
    plt.yticks(fontsize = 25)
    
    plt.xlabel('Time in Years', fontsize = 25)
    plt.ylabel('Total Emissions', fontsize = 25)
    plt.legend(loc='best', fontsize = 25)
    plt.title('Rolling Mean & Standard Deviation', fontsize = 25)
    plt.show(block= True)

정상성 체크 후 주요 통계값 출력해주는 함수 정의

In [ ]:
def Adfuller(ts, cutoff = 0.01):
    ts_test = adfuller(ts, autolag = 'AIC')
    ts_test_output = pd.Series(ts_test[0:4], index=['Test Statistic',
                                                    'p-value',
                                                    '#Lags Used',
                                                    'Number of Observations Used'])
    
    for key,value in ts_test[4].items():
        ts_test_output['Critical Value (%s)'%key] = value
    print(ts_test_output)
    
    if ts_test[1] <= cutoff:
        print("Strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root, hence it is stationary")
    else:
        print("Weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

PM2.5(초미세먼지) 분석

In [ ]:
df_109 = pd.DataFrame(df_new.loc[(df_new['Station code']==109)])
df_109 = df_109.set_index("Measurement date")
df_25 = df_109.iloc[:,-1:]
df_25

In [ ]:
Plot(df_25)
Adfuller(df_25)

p값이 원하는 값이 아니라 차분

In [ ]:
first_difference = df_25 - df_25.shift(1)  
Plot(first_difference.dropna(inplace=False))
Adfuller(first_difference.dropna(inplace=False))

계절성 체크

In [ ]:
Adfuller(seasonal_first_difference.dropna(inplace=False))
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(seasonal_first_difference.iloc[13:], lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(seasonal_first_difference.iloc[13:], lags=40, ax=ax2)

p,d,q값을 해결하기 위해 여러 작업


In [ ]:
p = d = q = range(0, 2) # 이 값들은 0~2사이의 값
pdq = list(itertools.product(p, d, q))
pdq_x_QDQs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of Seasonal ARIMA parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], pdq_x_QDQs[1]))
print('SARIMAX: {} x {}'.format(pdq[2], pdq_x_QDQs[2]))

In [ ]:
start_day = '2017-01-01'
end_day = '2019-12-31'
con1=df_25.index>=start_day
con2=df_25.index<=end_day
df_25_train=df_25[con1&con2]

아래는 해당 p,d,q값을 일일히 for문을 돌려서 AIC값을 출력하고, 가장 낮은 값을 채택해서 아리마 모형에 적용하기 위해 사용

In [ ]:
warnings.filterwarnings("ignore")
for param in pdq:
    for param_seasonal in pdq_x_QDQs:
        try:
            mod = sm.tsa.statespace.SARIMAX(df_25_train,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

데이터 training

In [ ]:
mod = sm.tsa.statespace.SARIMAX(df_25_train,
                                order=(1, 1, 1),
                                seasonal_order=(1, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

print(results.summary().tables[1])

잔차 분석

In [ ]:
results.resid.plot()

In [ ]:
print(results.resid.describe())

In [ ]:
results.plot_diagnostics(figsize=(15, 12))
plt.show()

예측값 출력

In [ ]:
pred = results.get_prediction(start=pd.to_datetime('2019-12-01'), dynamic=False)
pred_ci = pred.conf_int()

ax = df_25['2017-01':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('Date')
ax.set_ylabel('PM2.5 Levels')
plt.legend()

plt.show()

In [ ]:
y_forecasted = pred.predicted_mean
y_forecasted = pd.DataFrame(y_forecasted,columns={"PM2.5"})
y_truth = df_25_train['2019-12-01':]

mse = ((y_forecasted - y_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

pred_dynamic = results.get_prediction(start=pd.to_datetime('2019-12-01'), dynamic=True, full_results=True)
pred_dynamic_ci = pred_dynamic.conf_int()
pred_dynamic.predicted_mean

ax =  df_25_train['2019-10':].plot(label='observed', figsize=(20, 15))
pred_dynamic.predicted_mean.plot(label='Dynamic Forecast', ax=ax)

ax.fill_between(pred_dynamic_ci.index,
                pred_dynamic_ci.iloc[:, 0],
                pred_dynamic_ci.iloc[:, 1], color='k', alpha=.25)

ax.fill_betweenx(ax.get_ylim(), pd.to_datetime('2019-12-01'), df_25_train.index[-1],
                 alpha=.1, zorder=-1)

ax.set_xlabel('Date')
ax.set_ylabel('CO2 Levels')

plt.legend()
plt.show()

y_forecasted = pred_dynamic.predicted_mean
y_forecasted = pd.DataFrame(y_forecasted,columns={"PM2.5"})
mte_truth = df_25_train['2019-12-1':]

mse = ((y_forecasted - y_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

pred_uc = results.get_forecast(steps=500)

pred_ci = pred_uc.conf_int()